In [1]:
#Pandas è una libreria software scritta per il linguaggio di programmazione Python per la manipolazione e l'analisi dei dati. In particolare, offre strutture dati e operazioni per manipolare tabelle numeriche e serie temporali.
import pandas as pd
#NumPy è una libreria open source per il linguaggio di programmazione Python, che aggiunge supporto a grandi matrici e array multidimensionali insieme a una vasta collezione di funzioni matematiche di alto livello per poter operare efficientemente su queste strutture dati.
import numpy as np
#Matplotlib è una libreria per la creazione di grafici per il linguaggio di programmazione Python e la libreria matematica NumPy. Fornisce API orientate agli oggetti che permettono di inserire grafici all'interno di applicativi usando toolkit GUI generici, come WxPython, Qt o GTK.
import matplotlib.pyplot as plt
#Importo il modello di regressione lineare
#Il nostro obiettivo è confrontare la funzione scritta da noi con una implementata da una libreria (sklearn). Quindi prima la implementiamo manualmente e poi la confrontiamo con quella di sklearn.
from sklearn.linear_model import LinearRegression
# Modulo contenente Axes3D, un oggetto che può tracciare oggetti 3D su una figura matplotlib 2D.
from mpl_toolkits.mplot3d import axes3d
import time

### Introduzione
In questa parte, implementeremo la regressione lineare con più variabili per prevedere i prezzi delle case. Supponiamo che stai vendendo la tua casa e vuoi sapere quale sarebbe un buon prezzo di mercato. Un modo per farlo è quello di raccogliere prima informazioni sulle case recenti vendute e creare un modello dei prezzi delle case. Il set di dati contiene un set di formazione sui prezzi delle abitazioni a Portland, Oregon. La prima colonna è la dimensione della casa (in piedi quadrati), la seconda colonna è il numero di camere da letto e la terza colonna è il prezzo della casa.

In [2]:
#Carico i dati contenuti nel file che saranno splittati dal delimitatore ,
#Il dataset contiene i dati della popolazione in migliaia ed il profitto
data = np.loadtxt('data/ex1data2.txt', delimiter=',')

#Stampo i primi 10 elementi e tutte le colonne
print(data[:5,:])

#Stampo lo shape dei dati, cioè le dimensioni della struttura dati
print(data.shape)
#Prima colonna dimensioni casa in feat
#Seconda colonna numero di stanze da letto per ogni casa
#Terza colonna prezzo casa

[[2.104e+03 3.000e+00 3.999e+05]
 [1.600e+03 3.000e+00 3.299e+05]
 [2.400e+03 3.000e+00 3.690e+05]
 [1.416e+03 2.000e+00 2.320e+05]
 [3.000e+03 4.000e+00 5.399e+05]]
(47, 3)


## Apprendimento Supervisionato

In [3]:
X = data[:, :2]
y = data[:, -1]

m = X.shape[0]
n = X.shape[1]

#Passo dall' n-dimensionale alla matrice m x 1
#np.reshape(y, (y.shape[0], 1)) equivale a np.reshape(-1,1)
y = np.reshape(y, (y.shape[0], 1))

#print("Training examples : {}".format(m))
#print("Features : {}".format(n))

#print("First 10 examples: \n", X[:10, :])

In [4]:
for i in range(m):
    print("{}-th example; Input features {} Target value {}".format((i+1), X[i], y[i]))

1-th example; Input features [2104.    3.] Target value [399900.]
2-th example; Input features [1600.    3.] Target value [329900.]
3-th example; Input features [2400.    3.] Target value [369000.]
4-th example; Input features [1416.    2.] Target value [232000.]
5-th example; Input features [3000.    4.] Target value [539900.]
6-th example; Input features [1985.    4.] Target value [299900.]
7-th example; Input features [1534.    3.] Target value [314900.]
8-th example; Input features [1427.    3.] Target value [198999.]
9-th example; Input features [1380.    3.] Target value [212000.]
10-th example; Input features [1494.    3.] Target value [242500.]
11-th example; Input features [1940.    4.] Target value [239999.]
12-th example; Input features [2000.    3.] Target value [347000.]
13-th example; Input features [1890.    3.] Target value [329999.]
14-th example; Input features [4478.    5.] Target value [699900.]
15-th example; Input features [1268.    3.] Target value [259900.]
16-t

## Scaling delle features (normalizzazione in forma di scaling)

Poichè le features differiscono in ordine di grandezza bisogna effettuare uno scaling che permette una più veloce convergenza
con il gradient descent. La funzione featureNormalize() esegue le seguenti procedure:
    1) Sottrae il valore medio di ogni feature dal dataset;
    2) Dopo aver sottratto la media, ridimensionare ulteriormente (scale o divide) le features values per la rispettiva deviazione standard. 
    La deviazione standard è un modo per misurare quanta variazione ci sia nell'intervallo di valori di una particolare 
    caratteristica (la maggior parte dei punti di dati si troveranno entro +-2 della deviazione standard della media); questa 
    è un'alternativa alla presa dell'intervallo di valori (max-min). Nel momento in cui viene chiamato featureNormalize(), 
    la colonna aggiuntiva di 1 corrispondente a x0 = 1 non è stata ancora aggiunta a X.
    
### Note implementative
Durante la normalizzazione delle funzioni, è importante memorizzare i valori utilizzati per la normalizzazione: il valore medio e la deviazione standard utilizzata per i calcoli. Dopo aver appreso i parametri dal modello, spesso vogliamo prevedere i prezzi delle case che non abbiamo mai visto prima. Dato un nuovo valore x (area soggiorno e numero di camere da letto), dobbiamo prima normalizzare x usando la deviazione media e standard precedentemente calcolata dal set di addestramento.

$$ \large i = 0,...,m $$
$$ \large j = 0,...,n $$
$$ \large x_{j}^{(i)} = \frac{x_{j}^{(i)}-\mu_{j}}{\sigma_{j}} $$
$$ \large x_{j}^{(i)} = \frac{x_{j}^{(i)}-\mu_{j}}{max_{j} - min_{j}} $$
$$ \sigma->(deviazione-standard) $$ 
$$ \mu->(media) $$ 
$$ \max->(max) $$
$$ \min->(min) $$
$$ j->colonna $$
$$ i->i-esimo-training-example $$

In [5]:
#Accetto in input X
def featureScaling(X):
    X_scaled = X
    
    #Inizializzo queste 4 variabili ai valori di media, ds, max e min; le quali hanno 1 riga ed il numero di colonne pari ad X (2 nel nostro caso)
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    maxima = np.amax(X, axis=0)
    minima = np.amin(X, axis=0)

    for j in range(X.shape[1]):
        X_scaled[:,j] = (X[:,j] - mu[j])/sigma[j]
        #X_scaled[:,j] = (X[:,j] - mu[j])/(maxima[j] - minima[j])
        
    return X_scaled, mu, sigma, maxima, minima

### N.B. Bisogna prima effettuare lo scaling e poi aggiungere la colonna di 1 per pesare le features

### Ridimensione l'input X

In [6]:
X_scaled, mu_X, sigma_X, maxima_X, minima_X = featureScaling(np.copy(X))

print("Mean : {}".format(mu_X))
print("Standard Deviation : {}".format(sigma_X))
print("Max : {}".format(maxima_X))
print("Min : {}".format(minima_X))

[[2.104e+03 3.000e+00]
 [1.600e+03 3.000e+00]
 [2.400e+03 3.000e+00]
 [1.416e+03 2.000e+00]
 [3.000e+03 4.000e+00]
 [1.985e+03 4.000e+00]
 [1.534e+03 3.000e+00]
 [1.427e+03 3.000e+00]
 [1.380e+03 3.000e+00]
 [1.494e+03 3.000e+00]
 [1.940e+03 4.000e+00]
 [2.000e+03 3.000e+00]
 [1.890e+03 3.000e+00]
 [4.478e+03 5.000e+00]
 [1.268e+03 3.000e+00]
 [2.300e+03 4.000e+00]
 [1.320e+03 2.000e+00]
 [1.236e+03 3.000e+00]
 [2.609e+03 4.000e+00]
 [3.031e+03 4.000e+00]
 [1.767e+03 3.000e+00]
 [1.888e+03 2.000e+00]
 [1.604e+03 3.000e+00]
 [1.962e+03 4.000e+00]
 [3.890e+03 3.000e+00]
 [1.100e+03 3.000e+00]
 [1.458e+03 3.000e+00]
 [2.526e+03 3.000e+00]
 [2.200e+03 3.000e+00]
 [2.637e+03 3.000e+00]
 [1.839e+03 2.000e+00]
 [1.000e+03 1.000e+00]
 [2.040e+03 4.000e+00]
 [3.137e+03 3.000e+00]
 [1.811e+03 4.000e+00]
 [1.437e+03 3.000e+00]
 [1.239e+03 3.000e+00]
 [2.132e+03 4.000e+00]
 [4.215e+03 4.000e+00]
 [2.162e+03 4.000e+00]
 [1.664e+03 2.000e+00]
 [2.238e+03 3.000e+00]
 [2.567e+03 4.000e+00]
 [1.200e+03

Ci sono due metodologie per trovare il modello migliore valuntando il costo ed aggiornando i pesi:
    1)Con il Gradient Descent per il quale abbiamo bisogno di scegliere l'iperparametro alfa e settare un grande numero di iterazioni.
    Il GD è da scegliere quando è presente un grande numero di features, 10^6 è un numero ragionevole di features.
    2)Con la Normal Equation per la quale non c'è bisogno di scegliere alfa e non sono presenti iterazioni visto il calcolo one shot sui
    pesi per poi valutare la Cost function. Per un numero di features elevato il costo computazionale risulta essere troppo elevato, 
    meglio se usata con piccoli dataset 100 o 1000 features.
N.B. Nella Normal Equation c'è bisogno che X^T * X sia invertibile, essa non lo è quando abbiamo features ridondanti o quando si hanno 
molte features rispetto ai training examples (la Regularization ovvia tale problema).

## Ridimensiono l'output y (Solo per scopo visualizzativo)

In [7]:
y_scaled, mu_y, sigma_y, maxima_y, minima_y = featureScaling(np.copy(y))

print("Mean : {}".format(mu_y))
print("Standard Deviation : {}".format(sigma_y))
print("Max : {}".format(maxima_y))
print("Min : {}".format(minima_y))

Mean : [340412.65957447]
Standard Deviation : [123702.53600615]
Max : [699900.]
Min : [169900.]


### Cost Function (Vectorized)

In [8]:
#Creo la funzione di costo
#X input
#y = output
#Se non passo esplicitamente theta, theta è settata di default come un vettore nullo (di 0) di 2 righe ed una colonna
def computeCostVectorized(X, y, theta=np.zeros((X.shape[1],1))):
    #Calcolo il numero di examples
    m = y.size
    
    #Setto una variabile per la funzione di costo
    J = 0
    
    start = time.time()
    h = X.dot(theta)
    J = 1/(2*m)*((h-y).T.dot(h-y))
    end = time.time()
    eta = end -start
    
    return(J, eta)

#Il tempo rappresentato sotto è il tempo trascorso per eseguire la cella

### Cost Function (Loop-based Numpy)

In [9]:
def computeCostLoopNumpy(X, y, theta=np.zeros((X.shape[1], 1))):
    m = y.size
    J = 0
    h = np.zeros((m,1))
    
    start = time.time()
    
    # hypothesis evaluation loop-based implementation
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            #h[i] = h[i] + theta[j] * X[i,j]
            h[i] += theta[j] * X[i,j]
    
    # semi-vectorized implementation (Numpy)
    J = 1/(2*m)*np.sum(np.square(h-y))
    end = time.time()
    eta = end-start
    return(J, eta)

In [10]:
#Passo X.shape[1] in modo da non vincolare la grandezza sul numero di colonne
computeCostVectorized(X,y,theta=np.zeros((X.shape[1],1)))

(array([[6.55915481e+10]]), 0.0005841255187988281)

### Linear Regression Hypothesis for Contour plots 
$$ \theta_{0}=0 $$
$$ h_{0}(x)=\theta_{1}x_{1} + \theta_{1}x_{2} $$
Sacrifichiamo theta0 per avere un plot capibile, altrimenti i plot sarebbero
3D e difficili da comprendere.

### Contour Plots (con theta1 e theta0)

In [ ]:
B1 = np.linspace(-500, 700, 100)
B2 = np.linspace(-500000, 500000, 100)

BS1 = np.linspace(-10, 10, 100)
BS2 = np.linspace(-10, 10, 100)

#Creo la griglia bidimensionale indicizzata xy
xx, yy = np.meshgrid(B1, B2, indexing='xy')
xxs, yys = np.meshgrid(BS1, BS2, indexing='xy')
Z = np.zeros((B1.size, B2.size))
Z_scaled = np.zeros((BS1.size, BS2.size))

for(i,j),v in np.ndenumerate(Z):
    Z[i,j], _ = computeCostVectorized(X,y,[[xx[i,j]],[yy[i,j]]])
    
for(i,j),v in np.ndenumerate(Z):
    Z_scaled[i,j], _ = computeCostVectorized(X_scaled,y_scaled,[[xxs[i,j]],[yys[i,j]]])
    
    
fig = plt.figure(figsize=(30,10))
ax1=fig.add_subplot(121)
ax2=fig.add_subplot(122)

#Logspace Restituisce uniformemente gli spazi numerici con intervallo w.r.t su una log scale.
CS_1 = ax1.contour(xx, yy, Z, np.logspace(-300,300,3000), cmap=plt.cm.jet, alpha=0.6)
ax1.set_title('Contour Plot (Without Feature Scaling)')

CS_2 = ax2.contour(xx, yy, Z_scaled, np.logspace(-100,300,3000), cmap=plt.cm.jet, alpha=0.6)
ax2.set_title('Contour Plot (With Feature Scaling)')

for ax in fig.axes:
    ax.set_xlabel(r'$\theta_1$', fontsize=17)
    ax.set_ylabel(r'$\theta_2$', fontsize=17)

### Gradient Descent Vectorized
y non viene scalato quando si usa il Gradient Descent

### Multivariate Linear Regression Hypothesis
$$ \large h_{0}(x)=\theta_{0} + \theta_{1}x_{1} + \theta_{1}x_{2} $$

### Aggiungo la colonna di uno che pesano il parametro theta0

In [ ]:
X = np.c_[np.ones((X.shape[0], 1)), X]
X_scaled = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled]

In [ ]:
print(X.shape)
print(X_scaled.shape)

### Gradient Descent
L'unica differenza con la linear regression è che c'è un'altro example nella matrice X. La funzione di ipotesi e la regola di aggiornamento della discesa del gradiente batch rimangono invariate. Il modo più intelligente per affrontare questo problema è guardare all'implementazione vettoriale della discesa del gradiente, che è indipendente dal numero di features. Ricorda di aggiungere la colonna di 1 per il termine di intercetta.

**Vectorized Implementation of the cost function**

$$\large J(\theta) = \frac{1}{2m} (X\theta - y)^T(X\theta - y)$$

**Vectorized Implementation of Gradient Descent** 
$$\large \theta = \theta - \frac{\alpha}{m} ((X\theta - y)^T X)^T = \theta - \frac{\alpha}{m} (X^T(X\theta - y)$$

In [ ]:
def gradientDescentVectorized(X, y, theta=np.zeros((X.shape[1],1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters, dtype = 'float64')
    
    start = time.time()
    for iter in np.arange(num_iters):
        h = X.dot(theta)

        # ! simoultaneusly update all the parameters 
        # vectorized implementation
        theta = theta - alpha*(1.0/m)*(X.T.dot(h-y))
        J_history[iter], _ = computeCostVectorized(X, y, theta)

        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break

    end = time.time()
    eta = end - start
    return(theta.ravel(), J_history[J_history != 0], eta)

In [ ]:
def gradientDescentLoopNumpy(X, y, theta=np.zeros((X.shape[1],1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters)
    
    start = time.time()
    for iter in np.arange(num_iters):
        
        h = np.zeros((m,1))
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                h[i] += theta[j] * X[i,j]

        # ! simoultaneusly update all the parameters
        for j in range(X.shape[1]):
            gradient = 0
            for i in range(X.shape[0]):
                gradient += (h[i]-y[i])*X[i,j]
            theta[j] = theta[j] - alpha*(1/m)*gradient

        J_history[iter], _ = computeCostLoopNumpy(X, y, theta)
        
        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break
            
    end = time.time()
    eta = end - start
    return(theta.ravel(), J_history[J_history != 0], eta)

## Addestrare il modello di Multivariate Linear Regression con e senza Feature Scaling

In [ ]:
print(X.shape[1])
theta = np.zeros((X.shape[1], 1))
theta[0] = 0
theta[1] = 160
theta[2] = 20000

alpha = 0.01
num_iters = 10000

theta_wfs, Cost_J_wfs, eta_wfs = gradientDescentVectorized(X, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = False)
theta_fs, Cost_J_fs, eta_fs = gradientDescentVectorized(X_scaled, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = False)

theta_wfs_np, Cost_J_wfs_np, eta_wfs_np = gradientDescentLoopNumpy(X, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = False)
theta_fs_np, Cost_J_fs_np, eta_fs_np = gradientDescentLoopNumpy(X_scaled, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = False)

In [ ]:
print("theta values found by GD Vectorized without feature scaling : {}".format(theta_wfs))
print("theta values found by GD Vectorized with feature scaling : {}".format(theta_fs))
print("theta values found by GD Loop-based Numpy without feature scaling : {}".format(theta_wfs_np))
print("theta values found by GD Loop-based Numpy with feature scaling : {}".format(theta_fs_np))

# Optimal Value of theta : [340412.6679957  109447.78994015  -6578.35510528]
# I valori nan sono causati da un'overflow poichè i valori non sono normalizzati o standardizzati

In [ ]:
fig = plt.figure(figsize = (15,8))
ax = fig.add_subplot(111, projection='3d')

X_tmp = np.sort(np.c_[X_scaled[:,1], X_scaled[:,2], X_scaled.dot(theta_fs)], axis = 0)


ax.scatter(X_scaled[:,1], X_scaled[:,2], y, c = 'r', label = 'training data')
#ax.plot(X_scaled[:,1], X_scaled[:,2], X_scaled.dot(theta_fs), label = 'multivariate linear model')
ax.plot(X_tmp[:,0], X_tmp[:,1], X_tmp[:,2], label = 'multivariate linear model')

ax.set_xlabel('Size (in squared feet)')
ax.set_ylabel('Number of bedrooms')
ax.set_zlabel('Price')
ax.legend()

ax.set_title('Housing Prices in Portland (Oregon) - Multivariate Analysis', y = 1.1)
plt.show()

## Trovo $\theta^{'}$ per gli input non scalati (closed form)

$$\large h_{\theta}(x^{(i)}) = \theta_{0} + \theta_{1}x_{1}^{(i)} + \theta_{2}x_{2}^{(i)}$$

$$\large h_{\theta}(x^{(i)}) = \theta_{0} + \theta_{1}\frac{x_{1}^{(i)}-\mu_1}{\sigma_1} + \theta_{2}\frac{x_{2}^{(i)}-\mu_2}{\sigma_2}$$

$$\large \sigma_1 \sigma_2 h_{\theta}(x^{(i)}) = \sigma_1 \sigma_2 \theta_{0} + \theta_{1}\sigma_{2}(x_{1}^{(i)}-\mu_1) + \theta_{2}\sigma_{1}(x_{2}^{(i)}-\mu_2)$$

$$\large \sigma_1 \sigma_2 h_{\theta}(x^{(i)}) = \sigma_1 \sigma_2 \theta_{0} + \theta_{1}\sigma_{2}x_{1}^{(i)}-\theta_{1}\sigma_{2}\mu_1 + \theta_{2}\sigma_{1}x_{2}^{(i)}-\theta_{2}\sigma_{1}\mu_2$$

$$\large \sigma_1 \sigma_2 h_{\theta}(x^{(i)}) = (\sigma_1 \sigma_2 \theta_{0} -\theta_{1}\sigma_{2}\mu_1 -\theta_{2}\sigma_{1}\mu_2) + \theta_{1}\sigma_{2}x_{1}^{(i)} + \theta_{2}\sigma_{1}x_{2}^{(i)}$$

$$\large h_{\theta}(x^{(i)}) = \frac{(\sigma_1 \sigma_2 \theta_{0} -\theta_{1}\sigma_{2}\mu_1 -\theta_{2}\sigma_{1}\mu_2)}{\sigma_1\sigma_2} + \frac{\theta_{1}x_{1}^{(i)}}{\sigma_1} + \frac{\theta_{2}x_{2}^{(i)}}{\sigma_{2}}$$

$$\large h_{\theta^{'}}(x^{(i)}) = \theta_{0}^{'} + \theta_{1}^{'}x_{1}^{(i)} + \theta_{2}^{'}x_{2}^{(i)}$$

$$\begin{equation}\left\{\begin{aligned}&\large \theta_{0}^{'} = \frac{(\sigma_1 \sigma_2 \theta_{0} -\theta_{1}\sigma_{2}\mu_1 -\theta_{2}\sigma_{1}\mu_2)}{\sigma_1\sigma_2} \\&\large \theta_{1}^{'} = \frac{\theta_{1}}{\sigma_1} \\&\large \theta_{2}^{'} = \frac{\theta_{2}}{\sigma_2}\end{aligned}\right.\end{equation}$$

### General formulation
$$\large h_{\theta}(x^{(i)}) = \theta_{0} + \theta_{1}\frac{x_1^{(i)}-\mu_1}{\sigma_1} + \theta_{2}\frac{x_2^{(i)}-\mu_2}{\sigma_2} + \dots + \theta_{n}\frac{x_n^{(i)}-\mu_n}{\sigma_n} $$

$$ h_{\theta}(x^{(i)}) = \frac{[\theta_{0}(\sigma_1\sigma_2\sigma_3\dots\sigma_n) + \theta_{1}(\sigma_2\sigma_3\dots\sigma_n)(x_1^{(i)}-\mu_1) + \theta_{2}(\sigma_1\sigma_3\dots\sigma_n)(x_2^{(i)}-\mu_2) + \dots + \theta_{n}(\sigma_1\sigma_2\dots\sigma_{n-1})(x_n^{(i)}-\mu_n)]}{\sigma_1\sigma_2\sigma_3\dots\sigma_n} $$

$$ h_{\theta}(x^{(i)}) = \frac{[\theta_{0}(\sigma_1\sigma_2\sigma_3\dots\sigma_n) - \theta_1(\sigma_2\sigma_3\dots\sigma_n)\mu_1 - \theta_2(\sigma_1\sigma_3\dots\sigma_n)\mu_2 + \dots - \theta_n(\sigma_1\sigma_2\dots\sigma_{n-1})\mu_n + \theta_1(\sigma_2\sigma_3\dots\sigma_n)x_1^{(i)} + \theta_2(\sigma_1\sigma_3\dots\sigma_n)x_2^{(i)} + \dots + \theta_n(\sigma_1\sigma_2\dots\sigma_{n-1})x_n^{(i)}]}{\sigma_1\sigma_2\sigma_3\dots\sigma_n} $$

$$\left\{\begin{aligned}&\large h_{\theta^\prime}(x^{(i)}) = \theta_{0}^\prime + \theta_{1}^\prime x_{1}^{(i)} + \theta_{2}^\prime x_{2}^{(i)} + \dots + \theta_{n}^\prime x_{n}^{(i)} \\ &\large \theta_0^\prime = \frac{\theta_0\prod_{i=1}^n\sigma_i -\sum_{i=1}^n{\theta_i\mu_i\prod_{j\neq i}^n\sigma_j}}{\prod_{i=1}^n\sigma_i} = \theta_0 - \sum_{i=1}^n\frac{\theta_i}{\sigma_i}\mu_i = \theta_0 - \sum_{i=1}^n\theta_i^\prime\mu_i \\ &\large \theta_i^\prime = \frac{\theta_i}{\sigma_i},\ \forall i = 1,\dots,n\end{aligned}\right.$$



## Eseguo una predizione su un example mai visto

In [ ]:
#Output senza scaling
x_tmp = np.array([1,1650,3])
yhat_tmp = np.dot(x_tmp,theta_fs)
print('yhat tmp : {}'.format(yhat_tmp))
# Estimate the price of a 1650 sq-ft, 3 br house
# Remember to scale the new input
#Testiamo il modello con nuovi valori
x = np.array([1650, 3])
#Tali nuovi valori devono essere scalati
x = np.array([1, (x[0]-mu_X[0])/sigma_X[0], (x[1]-mu_X[1])/sigma_X[1]]).reshape(1,-1)

#Eseguo il prodotto vettoriale tra i parametri theta del modello trovati ed i nuovi valori passati
#x ha dimensione 1x3 e theta_fs di 3x1, quindi ottengo uno scalare di dimensione 1x1
yhat_scaled = np.dot(x, theta_fs) 
print('Case with Feature Scaling : Predicted price of a 1650 sq-ft, 3 br house (using gradient descent): {}'.format(yhat_scaled))

#Poichè i theta sono scalati non posso applicare i nuovi dati della casa direttamente ma bisogna riportare i valori di theta scalati a quelli normali
#per poi eseguire la predizione
x1 = np.array([1,1650,3])
thetap = np.copy(theta_fs)
thetap[0] = (sigma_X[0]*sigma_X[1]*theta_fs[0] - theta_fs[1]*sigma_X[1]*mu_X[0] - theta_fs[2]*sigma_X[0]*mu_X[1])/(sigma_X[0]*sigma_X[1])
thetap[1] = theta_fs[1]/sigma_X[0]
thetap[2] = theta_fs[2]/sigma_X[1]
yhat = np.dot(x1, thetap) 
print('Case without Feature Scaling : Predicted price of a 1650 sq-ft, 3 br house (using gradient descent): {}'.format(yhat))


In [ ]:
print("theta found by GD Vectorized with Feature Scaling : {}".format(theta_fs))
print("theta found by the closed form (for unscaled input) : {}".format(thetap))


### Confrontiamo i valori $\theta^{'}$ con quelli trovati con Sklearn (Ordinary Least Squares)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# Compare with Scikit-learn Linear regression 
regr = LinearRegression()
regr.fit(X, y.ravel())

In [ ]:
print("theta found by Linear Regression Sklearn (OLS) (for unscaled input) : {}".format([regr.intercept_, regr.coef_[1], regr.coef_[2]]))

## Normal equation

Esiste una soluzione in forma chiusa alla regressione lineare. È possibile applicando l'equazione normale che porta ai valori dei parametri migliori senza iterazioni.
$$ \large \theta = (X^{T}X)^{-1}X^{T}y $$
$$ J(\theta) = \frac{1}{2m}\sum_{i=1}^{m} (h_{\theta}(x_{i})-y_{i})^{2} $$
$$ J(\theta) = \frac{1}{2m}(X\theta-y)^{T}(X\theta-y) $$
$$ = ((X\theta)^{T}-y^{T})(X\theta-y) $$
$$ = (X\theta)^{T}(X\theta)-(X\theta)^{T}y-y^{T}X\theta+y^{T}y $$
$$ = \theta^{T}X^{T}X\theta-2(X\theta)^{T}y+y^{T}y $$
$$ = \theta^{T}X^{T}X\theta-2\theta^{T}X^{T}y+y^{T}y $$
$$ \frac{\partial J(\theta)}{\partial \theta} = 2X^{T}X\theta-2X^{T}y=0 $$
$$ X^{T}X\theta=X^{T}y $$
$$ \theta=(X^{T}X)^{-1}X^{T}y $$
L'uso di questa formula non richiede alcun ridimensionamento delle features e si ottiene una soluzione esatta in un calcolo: non esiste un "ciclo fino alla convergenza" come nella discesa del gradiente. Ricorda che non c'è bisogno di ridimensionare le features, dobbiamo comunque aggiungere una colonna di 1 alla matrice X per avere un termine di intercetta theta0. NormalEquation() implementa la formula sopra.

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
#X è 47x3
#y è 47x1
#quindi theta è (3x47)(47x1)=3x1
def normalEquations(X,y):
    start = time.time()
    #Calcolo la pseudo inversa anzichè l'inversa poichè il problema potrebbe essere malcondizionato (cioè ho feature ridondanti)
    #oppure potrei avere più features rispetto i training examples; la pseudo inversa ovvia al problema stimando l'inversa. Essa è una matrice quadrata.
    #Viene fatta per eliminare i punti di singolarità
    pinv = np.linalg.pinv(X.T.dot(X))
    theta_ne = pinv.dot(X.T).dot(y)
    end = time.time()
    eta_ne = end - start
    return theta_ne, eta_ne

In [ ]:
theta_ne, eta_ne = normalEquations(X,y)
print(theta_ne)

In [ ]:
print("theta found by GD Vectorized with Feature Scaling : {}".format(theta_fs))
print("theta found by the closed form (for unscaled input) : {}".format(thetap))
print("theta found by Linear Regression Sklearn (OLS) (for unscaled input) : {}".format([regr.intercept_, regr.coef_[1], regr.coef_[2]]))
print("theta found by Normal Equation : {}".format(theta_ne))

### Execution Time Comparison

In [ ]:
theta = np.zeros((X.shape[1],1))
alpha = 0.01
num_iters = 10000

theta_vec_wfs, J_vec_wfs, eta_wfs_vec = gradientDescentVectorized(X, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = True)
theta_vec_wfs, J_vec_fs, eta_fs_vec = gradientDescentVectorized(X_scaled, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = True)

theta_lp_wfs_np, J_lp_wfs_np, eta_lp_wfs_np = gradientDescentLoopNumpy(X, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = True)
theta_lp_fs_np, J_lp_fs_np, eta_lp_fs_np = gradientDescentLoopNumpy(X_scaled, y, theta = np.copy(theta), alpha = alpha, num_iters = num_iters, early = True)

In [ ]:
print("The GD Vectorized without feature scaling stops at {} iterations".format(len(J_vec_wfs)))
print("The GD Vectorized with feature scaling stops at {} iterations".format(len(J_vec_fs)))
print("The GD Loop-based Numpy without feature scaling stops at {} iterations".format(len(J_lp_wfs_np)))
print("The GD Loop-based Numpy with feature scaling stops at {} iterations".format(len(J_lp_fs_np)))
print("")
print("Execution time GD Vectorized without features scaling until crash : {} [ms] {} [\u03BC]".format(round(eta_wfs_vec,6)*1000, round(eta_wfs_vec,6)*1000000))
print("Execution time GD Vectorized with features scaling : {} [ms] {} [\u03BC]".format(round(eta_fs_vec,6)*1000, round(eta_fs_vec,6)*1000000))
print("Execution time Normal Equation : {} [ms] {} [\u03BC]".format(round(eta_ne,6)*1000, round(eta_ne,6)*1000000))

In [ ]:
print("theta found by GD Vectorized with scaled input & early stopping is : {}".format(theta_vec_wfs))
#I theta della Normal Equation sono uguali a quelli in forma chiusa
print("theta found by Normal Equation is : {}".format([theta_ne[0][0],theta_ne[1][0],theta_ne[2][0]]))

#Verifichiamo che i theta trovati con il GD Vectorized, applicando le formule d'inversione mi ritornato quelli della Normal Equation
theta0p = (sigma_X[0]*sigma_X[1]*theta_vec_wfs[0] - theta_vec_wfs[1]*sigma_X[1]*mu_X[0] - theta_vec_wfs[2]*sigma_X[0]*mu_X[1])/(sigma_X[0]*sigma_X[1])
theta1p = theta_vec_wfs[1]/sigma_X[0]
theta2p = theta_vec_wfs[2]/sigma_X[1]
print("theta' found by formulas is : {}".format([theta0p, theta1p, theta2p]))

In [ ]:
x = np.array([1650,3])
x = np.array([1, (x[0]-mu_X[0])/sigma_X[0], (x[1]-mu_X[1])/sigma_X[1]])

x = x.reshape((1,x.shape[0]))
price = x.dot(theta_ne)
print('Predicted price of a 1650 sq-ft, 3 br house(using normal equations): {}'.format(price))

# DOMANDA: MEMORIZZARE LA NON INVERTIBILITA' SULL'INVERSA E LA PSEUDO INVERSA
#### In matematica, e in particolare in algebra lineare, la matrice pseudo-inversa, o pseudo-inversa di Moore-Penrose, di una matrice data A si indica con A^{+} ed è la generalizzazione della matrice inversa al caso in cui A non sia quadrata.

![title](data/Pseudo-inversa.png)